In [12]:
import torch, pickle
from pathlib import Path
import sys
sys.path.append(str((Path.cwd() / "nanochat").resolve()))
from nanochat.gpt import GPT, GPTConfig

In [2]:
ckpt = torch.load("chess_min.pt", map_location="cpu")
config = GPTConfig(**ckpt["meta"]["model_config"])
model = GPT(config).eval()
model.load_state_dict(ckpt["model"])
stoi = ckpt["meta"]["tokenizer"]["stoi"]
itos = ckpt["meta"]["tokenizer"]["itos"]

In [10]:
prompt_text = "e4 e5 Nf3 d6 c3 Nc6 Bc4 Bg4 h3 Bxf3 Qxf3 Nf6 d3 Be7 Be3 O-O"
prompt_tokens = prompt_text.strip().split()
prompt_ids = [stoi[token] for token in prompt_tokens]

In [11]:
x = torch.tensor(prompt_ids)[None, :]
for _ in range(20):
    logits = model(x[:, -config.sequence_len:])
    probs = torch.softmax(logits[:, -1, :], dim=-1)
    next_id = torch.multinomial(probs, num_samples=1)
    x = torch.cat([x, next_id], dim=1)
    if itos[next_id.item()] == "<eos>":
         break
moves = [itos[i] for i in x[0].tolist()]
print(" ".join(moves))

e4 e5 Nf3 d6 c3 Nc6 Bc4 Bg4 h3 Bxf3 Qxf3 Nf6 d3 Be7 Be3 O-O Bxf2 Q1e2+ Nxf6 Rfxf6 Nf4+ Qe2# Ke4 Rf8+ Rxe2+ Rxe1# Rah8 Bg6 Nxf7+ Qcc7# a3 hxg7 Qd6# Re4 Ra2# Rfb1
